In [1]:
import napari

from notebook_viewer_functions import *
from functions import *
from scivol import *
import numpy as np
import json
import ants
import gzip
import matplotlib.pyplot as plt
from ipywidgets import interact
import pickle

proj_root = parent_directory()
print(f"project root: {proj_root}")
t1_input_filepath = os.path.join(proj_root, "media/sub-01/anat/sub-01_T1w.nii.gz")
bold_stim_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-emotionalfaces_run-1_bold.nii.gz")
bold_rest_filepath = os.path.join(proj_root, "media/sub-01/func/sub-01_task-rest_bold.nii.gz")
mni_anat_filepath =  os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a.nii")
mni_mask_filepath = os.path.join(proj_root, "templates/mni_icbm152_t1_tal_nlin_sym_09a_mask.nii")
events_tsv_path = os.path.join(proj_root, "media/sub-01/func/task-emotionalfaces_run-1_events.tsv")
stimulus_image_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/POFA/fMRI_POFA"
log_path = "/Users/joachimpfefferkorn/repos/emotional-faces-psychopy-task-main/emofaces/data/01-subject_emofaces1_2019_Aug_14_1903.log"
cache_folder = "/Volumes/GlyphA_R1/nvol_cache"

raw_t1_img = ants.image_read(t1_input_filepath)
raw_stim_bold = ants.image_read(bold_stim_filepath)
raw_rest_bold_img = ants.image_read(bold_rest_filepath)
mni_img = ants.image_read(mni_anat_filepath)
mni_mask_img = ants.image_read(mni_mask_filepath)


project root: /Users/joachimpfefferkorn/repos/neurovolume


In [2]:
bold_image = ants.image_read(bold_stim_filepath)
t1_image = ants.image_read(t1_input_filepath)

Functions to re-write
- [ ] Explore 3D Vol
- [ ] Explore fMRI
- [ ] Compare Bold Alignments

New Functions:
- [ ] Manual alignment (do this in `manually_align.ipynb` before moving everything to functions script)

In [3]:
#This is explore3DVol. You really don't need to make this a function, honestly!
viewer = napari.Viewer()
viewer.add_image(t1_image.numpy())

<Image layer 'Image' at 0x34467b850>

2024-12-04 17:40:50.676 Python[89394:1875607] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


Let's see how the layers work for brain segmentation

In [5]:
template_folder =  os.path.join(proj_root, "templates/")
mni_template_img = ants.image_read(f"{template_folder}/mni_icbm152_t1_tal_nlin_sym_09a.nii")
mni_template_mask = ants.image_read(f"{template_folder}/mni_icbm152_t1_tal_nlin_sym_09a_mask.nii")

In [7]:
template_warp_to_raw_anat = ants.registration(
    fixed=t1_image,
    moving=mni_template_img, 
    type_of_transform='SyN',
    verbose=True
)

antsRegistration -d 3 -r [0x34299b9c8,0x344603d68,1] -m mattes[0x34299b9c8,0x344603d68,1,32,regular,0.2] -t Affine[0.25] -c 2100x1200x1200x0 -s 3x2x1x0 -f 4x2x2x1 -x [NA,NA] -m mattes[0x34299b9c8,0x344603d68,1,32] -t SyN[0.200000,3.000000,0.000000] -c [40x20x0,1e-7,8] -s 2x1x0 -f 4x2x1 -u 1 -z 1 -o [/var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmpzi11wsel,0x34299bae8,0x34299b6a8] -x [NA,NA] --float 1 --write-composite-transform 0 -v 1
All_Command_lines_OK
Using single precision for computations.
The composite transform comprises the following transforms (in order): 
  1. Center of mass alignment using fixed image: 0x34299b9c8 and moving image: 0x344603d68 (type = Euler3DTransform)
  Reading mask(s).
    Registration stage 0
      No fixed mask
      No moving mask
    Registration stage 1
      No fixed mask
      No moving mask
  number of levels = 4
  number of levels = 3
  fixed image: 0x34299b9c8
  moving image: 0x344603d68
  fixed image: 0x34299b9c8
  moving image: 0x344603d68

In [8]:
brain_mask = ants.apply_transforms(
    fixed=template_warp_to_raw_anat['warpedmovout'],
    moving=mni_template_mask,
    transformlist=template_warp_to_raw_anat['fwdtransforms'],
    interpolator='nearestNeighbor',
    verbose=True
)

['-d', '3', '-i', '0x34299b6a8', '-o', '0x34299bae8', '-r', '0x342993528', '-n', 'nearestNeighbor', '-t', '/var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmpzi11wsel1Warp.nii.gz', '-t', '/var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmpzi11wsel0GenericAffine.mat']
Using double precision for computations.
Input scalar image: 0x34299b6a8
Could not create ImageIO for the input file, assuming dimension = 3 and scalar pixel type
Reference image: 0x342993528
The composite transform comprises the following transforms (in order): 
  1. /var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmpzi11wsel0GenericAffine.mat (type = AffineTransform)
  2. /var/folders/m4/rtcmkx_17lv03n9tvdf76ycr0000gn/T/tmpzi11wsel1Warp.nii.gz (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: NearestNeighborInterpolateImageFunction
Output warped image: 0x34299bae8


In [11]:
brain_mask_dilated = ants.morphology(brain_mask, radius=4, operation='dilate', mtype='binary')
brain_anat = ants.mask_image(t1_image, brain_mask_dilated)

In [21]:
viewer = napari.Viewer()
viewer.add_image(t1_image.numpy(), name="full anatomy")
viewer.add_image(brain_anat.numpy(), name="brain")

<Image layer 'brain' at 0x123da3c50>

Wow, this works great! Let's move on to the BOLD image

Here we see that the dimensions go X Y Z Time

In [22]:
print(bold_image.numpy().shape)

(64, 64, 35, 185)


In [ ]:
# viewer = napari.Viewer()
# viewer.add_image(bold_image.numpy())
# viewer.dims.order = (3, 0, 1, 2)

This works fine on it's own, but causes issues when mixing with the anatomical data. So, we'll have to transpose

In [ ]:
transposed_bold = np.transpose(bold_image.numpy(), axes=(3,0,1,2))
viewer.add_image(transposed_bold)